# GeoSpatial Public Policy Analysis - PyCon 2020
#### A repository containing the materials for the tutorial Geospatial Public Policy Analysis with GeoPandas for PyCon 2020. To be delivered virtually.

<details>
    <summary><strong>Goal</strong></summary>
    The goal of this notebook is to process NCES IPEDS tabular data into an analytical file (subset) that will be used in another notebook to visualize education deserts. 
    <ul>
        <li> Measurable goals for this notebook:</li>
        <li> 1. Identify <i>variables of interest</i> in dataset <strong>processed_data.csv</strong></li>
        <li> 2: Create a <i>working dataset</i> from dataset containing those variables of interest.</li>
    </ul>
</details>

<details>
    <summary><strong>Context</strong></summary>
    We've downloaded raw data from NCES IPEDS <a href="https://nces.ed.gov/ipeds/use-the-data/"><strong>https://nces.ed.gov/ipeds/use-the-data/</strong></a>. It has been pre-processed slightly for the purposes of this workshop. 
    <li>This notebook will create an analytical file to be used in the 01_Data_Visualization notebook.</li>
    <li>The pre-processed masterfile contains coordinates of higher education institutions as well as other institutional characteritics.</li>
    <li>The pre-processed mssterfile also contains graduation rate, demographic, and enrollment statistics.</li>
</details>

In [1]:
import pandas as pd
from pathlib import Path
from tools import tree
from datetime import datetime as dt
today = dt.today().strftime("%d-%b-%y")

today

'14-Apr-20'

In [2]:
RAW_DATA = Path("../data/raw/")
INTERIM_DATA = Path("../data/interim/")
PROCESSED_DATA = Path("../data/processed/")
FINAL_DATA = Path("../data/final/")
EXTERNAL_DATA = Path("../data/external/")

In [3]:
tree(PROCESSED_DATA)

+ ../data/processed
    + processed_data.csv


In [17]:
data = pd.read_csv(PROCESSED_DATA / 'processed_data.csv')
data.head().T

,0,1,2,3,4
unitid,180203,222178,138558,488031,172866
institution_name,Aaniiih Nakoda College,Abilene Christian University,Abraham Baldwin Agricultural College,Abraham Lincoln University,Academy College
fips_state_code,Montana,Texas,Georgia,California,Minnesota
sector,"Public, 2-year","Private not-for-profit, 4-year or above","Public, 4-year or above","Private for-profit, 4-year or above","Private for-profit, 4-year or above"
institutional_category,"Degree-granting, associate's and certificates ...","Degree-granting, primarily baccalaureate or above","Degree-granting, not primarily baccalaureate o...","Degree-granting, primarily baccalaureate or above","Degree-granting, not primarily baccalaureate o..."
street_address,269 Blackfeet Avenue Agency,,2802 Moore Hwy,100 West Broadway Suite 600,1600 W. 82nd Street Suite 100
institution_name_alias,,,ABAC,,
institution_name.1,Aaniiih Nakoda College,Abilene Christian University,Abraham Baldwin Agricultural College,Abraham Lincoln University,Academy College
longitude,-108.758,-99.7098,-83.5283,-118.257,-93.2998
latitude,48.4842,32.4689,31.4819,34.1463,44.8557


# Variables of interest

There is a lot of variables here. Some are demographic (i.e. `percent_total_enrollment_hispanic-latino`), others are related to graduation rates (`gradrate_*`) and others are institutional characteristics like the name, address, or state an institution is in.

To focus our research we need to choose one topic to tackle at a time. In this case we want to look at the equity implications of education deserts. We will revisit this dataset later to grab more charactersitics related to each higher education institution but for right now we only need the basics:
1. Name
2. Location (longitude and latitude, state)
3. Sector (public vs private, 2- vs 4-year)
4. Size (total enrollment, full- and part-time)

In [29]:
voi = [
    'institution_name',
    'longitude',
    'latitude',
    'fips_state_code',
    'sector',
    'total_enrollment',
    'full-time_enrollment',
    'part-time_enrollment',
]

In [30]:
data[voi].head()

,institution_name,longitude,latitude,fips_state_code,sector,total_enrollment,full-time_enrollment,part-time_enrollment
0,Aaniiih Nakoda College,-108.757816,48.484196,Montana,"Public, 2-year",150.0,108.0,42.0
1,Abilene Christian University,-99.709797,32.468943,Texas,"Private not-for-profit, 4-year or above",5204.0,3730.0,1474.0
2,Abraham Baldwin Agricultural College,-83.528281,31.481889,Georgia,"Public, 4-year or above",4291.0,2660.0,1631.0
3,Abraham Lincoln University,-118.256726,34.146294,California,"Private for-profit, 4-year or above",217.0,194.0,23.0
4,Academy College,-93.299809,44.855722,Minnesota,"Private for-profit, 4-year or above",99.0,81.0,18.0


In [31]:
working_data = data[voi].copy()

Now you'll notice sector is prividing actually 2 pieces of information: private vs public, 2- vs 4-year status. We should split those up.

In [32]:
working_data['sector'].str.split(',', expand = True)

,0,1
0,Public,2-year
1,Private not-for-profit,4-year or above
2,Public,4-year or above
3,Private for-profit,4-year or above
4,Private for-profit,4-year or above
...,...,...
3746,Public,4-year or above
3747,Private for-profit,2-year
3748,Private for-profit,2-year
3749,Public,2-year


We'll call those the `control` and `level` of the institution.

In [33]:
working_data['control'] = working_data['sector'].str.split(',', expand = True)[0]
working_data['level'] = working_data['sector'].str.split(',', expand = True)[1]

In [34]:
working_data.head()

,institution_name,longitude,latitude,fips_state_code,sector,total_enrollment,full-time_enrollment,part-time_enrollment,control,level
0,Aaniiih Nakoda College,-108.757816,48.484196,Montana,"Public, 2-year",150.0,108.0,42.0,Public,2-year
1,Abilene Christian University,-99.709797,32.468943,Texas,"Private not-for-profit, 4-year or above",5204.0,3730.0,1474.0,Private not-for-profit,4-year or above
2,Abraham Baldwin Agricultural College,-83.528281,31.481889,Georgia,"Public, 4-year or above",4291.0,2660.0,1631.0,Public,4-year or above
3,Abraham Lincoln University,-118.256726,34.146294,California,"Private for-profit, 4-year or above",217.0,194.0,23.0,Private for-profit,4-year or above
4,Academy College,-93.299809,44.855722,Minnesota,"Private for-profit, 4-year or above",99.0,81.0,18.0,Private for-profit,4-year or above


Now that we don't need the `sector` column we can `.drop` it.

In [35]:
working_data = working_data.drop(columns = ['sector'])

Note: you can also do 
```python
working_data.drop(columns = ['sector'], inplace = True)
```
but I once saw a talk by Marc Garcia (pandas core dev) where he said we shouldn't do that because some internal stuff in pandas. 🤷

In [36]:
working_data.head()

,institution_name,longitude,latitude,fips_state_code,total_enrollment,full-time_enrollment,part-time_enrollment,control,level
0,Aaniiih Nakoda College,-108.757816,48.484196,Montana,150.0,108.0,42.0,Public,2-year
1,Abilene Christian University,-99.709797,32.468943,Texas,5204.0,3730.0,1474.0,Private not-for-profit,4-year or above
2,Abraham Baldwin Agricultural College,-83.528281,31.481889,Georgia,4291.0,2660.0,1631.0,Public,4-year or above
3,Abraham Lincoln University,-118.256726,34.146294,California,217.0,194.0,23.0,Private for-profit,4-year or above
4,Academy College,-93.299809,44.855722,Minnesota,99.0,81.0,18.0,Private for-profit,4-year or above


***
Check-point!

We have all the necessary variables from this dataset. Let's save this subset for later use.

In [37]:
working_data.to_csv(PROCESSED_DATA / 'institutional_characteristics.csv', encoding = 'utf-8', index = False)